### Import necessary libraries

In [7]:
import os
import cv2
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical


2024-12-25 23:29:23.217886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735162163.239008    4921 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735162163.245519    4921 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 23:29:23.267466: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Variables

In [ ]:
classes = 43

### Create model

In [ ]:
def create_model(input_shape=(32, 32, 3), learning_rate=0.0001):
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),

        # Convolutional layer 1
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),

        # Pooling layer 1
        layers.MaxPooling2D((2,2)),

        # Convolutional layer 2
        layers.Conv2D(64, (3,3), activation='relu', input_shape=input_shape),
        
        # Pooling layer 2
        layers.MaxPooling2D((2,2)),
        
        # Convolutional layer 3
        layers.Conv2D(128, (3,3), activation='relu', input_shape=input_shape),
        
        # Flatten layer
        layers.Flatten(),
        
        # Dense layer 1
        layers.Dense(256, activation='relu'),
        
        # Dropout layer
        layers.Dropout(0.5),
        
        # Dense layer 2
        layers.Dense(43, activation='softmax')
    ])

    # Optimizer
    optimizer = keras.optimizers.Adam(learning_rate)
    
    # Loss
    loss = keras.losses.CategoricalCrossentropy()

    # Compile model
    model.compile(optimizer=optimizer, 
                  loss=loss,
                  metrics=['accuracy']
    )

    return model

### Resize all images to 32x32 pixels

In [144]:
def resize_images(size=(32, 32)):
    for i in range(classes):
        input_directory = f"../app/data/train/{i}"
        output_directory = f"../app/data/train-r/{i}"
        
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
    
        for filename in os.listdir(input_directory):
            input_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, filename)
    
            try:
                with Image.open(input_path) as img:
                    img = img.resize(size)
                    img.save(output_path)
                    print(f"Resized and saved: {output_path}")
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

In [ ]:
resize_images(size=(32, 32))

### Load train images

In [15]:
def load_images():
    
    sign_images = []
    sign_indexes = []

    for i in range(classes):
        
        input_directory = f"../app/data/train-r/{i}"
        
        for filename in os.listdir(input_directory):
            input_path = os.path.join(input_directory, filename)

            try:
                with Image.open(input_path) as img:
                    sign_images.append(np.array(img))
                    sign_indexes.append(i)
            except Exception as e:
                print(f"Error processing {input_path}: {e}")

    sign_images = np.array(sign_images)
    sign_indexes = np.array(sign_indexes)

    return sign_images, sign_indexes

In [16]:
sign_images, sign_indexes = load_images()

### Split train images

In [17]:
# Test folder already exists so we split the sign images into train and validation

X_train, X_val, y_train, y_val = train_test_split(sign_images, sign_indexes, test_size=0.2, random_state=42)

In [18]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [19]:
sign_images.shape, sign_indexes.shape

((39209, 32, 32, 3), (39209,))

In [20]:
X_train.shape, X_val.shape

((31367, 32, 32, 3), (7842, 32, 32, 3))

In [21]:
y_train.shape, y_val.shape

((31367, 43), (7842, 43))

### Train model

In [ ]:
model = create_model(input_shape=(32, 32, 3), learning_rate=0.0001)

In [ ]:
model.describe()

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint (
    "traffic_sign_classification_model.h5",
    save_best_only=True,
    monitor="val_accuracy"
)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=35,
                    batch_size=32,
                    callbacks=[checkpoint]
                   )

### Plot accuracy and loss

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

### Make predictions

In [ ]:
class_descriptions = {
    0: 'Speed limit (20km/h)',
    1: 'Speed limit (30km/h)',
    2: 'Speed limit (50km/h)',
    3: 'Speed limit (60km/h)',
    4: 'Speed limit (70km/h)',
    5: 'Speed limit (80km/h)',
    6: 'End of speed limit (80km/h)',
    7: 'Speed limit (100km/h)',
    8: 'Speed limit (120km/h)',
    9: 'No passing',
    10: 'No passing for vehicles over 3.5 metric tons',
    11: 'Right-of-way at the next intersection',
    12: 'Priority road',
    13: 'Yield',
    14: 'Stop',
    15: 'No vehicles',
    16: 'Vehicles over 3.5 metric tons prohibited',
    17: 'No entry',
    18: 'General caution',
    19: 'Dangerous curve to the left',
    20: 'Dangerous curve to the right',
    21: 'Double curve',
    22: 'Bumpy road',
    23: 'Slippery road',
    24: 'Road narrows on the right',
    25: 'Road work',
    26: 'Traffic signals',
    27: 'Pedestrians',
    28: 'Children crossing',
    29: 'Bicycles crossing',
    30: 'Beware of ice/snow',
    31: 'Wild animals crossing',
    32: 'End of all speed and passing limits',
    33: 'Turn right ahead',
    34: 'Turn left ahead',
    35: 'Ahead only',
    36: 'Go straight or right',
    37: 'Go straight or left',
    38: 'Keep right',
    39: 'Keep left',
    40: 'Roundabout mandatory',
    41: 'End of no passing',
    42: 'End of no passing by vehicles over 3.5 metric'
}

In [11]:
# This function gets a traffic sign image as an input and makes a prediction
def make_prediction(image):
    # Load model
    model = keras.models.load_model('./app/model/traffic_sign_classification_model.h5')
    
    # Load image
    img = load_img(image, target_size=(32,32))

    # Create image matrix
    x = np.array(img)
    X = np.array([x])
    print(type(x))
    print(X.shape)
    
    # Make prediction
    pred = model.predict(X)

    # Get class info
    class_index = pred[0].argmax()
    class_name = class_descriptions[class_index]

    # Return info
    return class_index, class_name

In [12]:
image = '../app/data/test/00000.png'

In [13]:
class_index, class_name = make_prediction(image)

<class 'numpy.ndarray'>
(1, 32, 32, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step


NameError: name 'class_descriptions' is not defined

In [101]:
class_index, class_name

(np.int64(16), 'Vehicles over 3.5 metric tons prohibited')